<a href="https://colab.research.google.com/github/adeepH/SPDRA-2021-SharedTask/blob/main/Notebooks/Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/adeepH/SPDRA-2021-SharedTask.git 

Cloning into 'SPDRA-2021-SharedTask'...
remote: Enumerating objects: 87, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 87 (delta 37), reused 6 (delta 2), pack-reused 0
Unpacking objects: 100% (87/87), done.


In [2]:
cd /content/SPDRA-2021-SharedTask/

/content/SPDRA-2021-SharedTask


In [3]:
!pip install -r requirements.txt

     |████████████████████████████████| 1.1MB 9.5MB/s 
     |████████████████████████████████| 1.1MB 25.5MB/s 
     |████████████████████████████████| 3.0MB 25.7MB/s 
     |████████████████████████████████| 890kB 40.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=7b148f582153df05bfe7f4239d84980442ca9cde747c106dff9b308d01e30097
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
cd scripts/

/content/SPDRA-2021-SharedTask/scripts


In [5]:
import pandas as pd
import numpy as np
import torch
from dataset import *
from model import *
from utils import *

In [6]:
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased') 
device = 'cuda'
MAX_LEN = 128
BATCH_SIZE = 32

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model = RFClassifier(7,'allenai/scibert_scivocab_uncased')
model.load_state_dict(torch.load('/content/drive/MyDrive/spdra2021/saved_models/scibert.bin'))
model.to(device)
for param in model.parameters():
    param.requires_grad = False
model.eval()

In [9]:
val = pd.read_csv('/content/drive/MyDrive/spdra2021/Datasets/validation.csv',delimiter=',',
                 header=None,names=['text','label'])

val.label = val.label.apply({'CL':0,'CR':1,'DC':2,
'DS':3,'LO':4, 'NI':5, 'SE':6}.get)

In [10]:
!python dataset.py

In [11]:
val_data_loader = create_data_loader(val,tokenizer,MAX_LEN,BATCH_SIZE)

In [12]:
y_review_texts, y_pred, y_pred_probs, y_test = get_predictions(
  model,
  val_data_loader
)

In [13]:
 
class_name = ['CL' ,'CR' ,'DC' ,'DS' ,'LO' , 'NI' , 'SE' ]
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test, y_pred, target_names=class_name,zero_division=0))

              precision    recall  f1-score   support

          CL       0.98      0.98      0.98      1866
          CR       0.92      0.91      0.91      1835
          DC       0.82      0.83      0.83      1355
          DS       0.93      0.93      0.93      1774
          LO       0.93      0.93      0.93      1217
          NI       0.91      0.91      0.91      1826
          SE       0.89      0.91      0.90      1327

    accuracy                           0.92     11200
   macro avg       0.91      0.91      0.91     11200
weighted avg       0.92      0.92      0.92     11200

